In [1]:
%autosave 0
%matplotlib inline
import tensorflow as tf
import math
import keras as kr
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp
#import seaborn as sb
#import pandas as pd
from keras.models import Sequential
import nnhealpix as nn
import nnhealpix.layers

Autosave disabled


Using TensorFlow backend.


## Création des $C_\ell$

In [3]:
l=np.linspace(5.0, 100.0, 2000)
l_p= (50.0-5.0)*np.random.random_sample(10010,) + 5
moy_l_p = np.mean(l_p)
ecart_l_p = np.sqrt(np.var(l_p))
sigma_p=5.0
print(l.shape, l_p.shape)
print(moy_l_p, ecart_l_p)

'print(l)\nprint(l_p)'

In [4]:
C_l=np.zeros((len(l),len(l_p)))
for i in range (len(l)):
    for j in range (len(l_p)):
        C_l[i,j]=np.exp(-((l[i]-l_p[j])**2.0)/(2.0*sigma_p**2.0))+10.0

(100, 100)


fig1=plt.figure(1,figsize=(10,10),edgecolor='black')
color=plt.colorbar()
color.set_label('$\ell _p$')
plt.ylabel('$C_\ell$')
plt.xlabel('Multipole $\ell$')
for i in range (len(l_p)):
    plt.plot(l,C_l)
plt.show

## Mapisation des $C_ \ell$

In [6]:
nside = 16
Maps = []
for j in range (len(l_p)) :
    Map = hp.sphtfunc.synfast(C_l[:,j], nside)
    Maps = np.append(Maps,Map)
Maps = Maps.reshape((12*nside**2, len(l_p)))
print(np.abs(Maps).max())
print(Maps.shape)

Sigma is 0.000000 arcmin (0.000000 rad) 
-> fwhm is 0.000000 arcmin
Sigma is 0.000000 arcmin (0.000000 rad) 
-> fwhm is 0.000000 arcmin
Sigma is 0.000000 arcmin (0.000000 rad) 
-> fwhm is 0.000000 arcmin
Sigma is 0.000000 arcmin (0.000000 rad) 
-> fwhm is 0.000000 arcmin
Sigma is 0.000000 arcmin (0.000000 rad) 
-> fwhm is 0.000000 arcmin
Sigma is 0.000000 arcmin (0.000000 rad) 
-> fwhm is 0.000000 arcmin
Sigma is 0.000000 arcmin (0.000000 rad) 
-> fwhm is 0.000000 arcmin
Sigma is 0.000000 arcmin (0.000000 rad) 
-> fwhm is 0.000000 arcmin
Sigma is 0.000000 arcmin (0.000000 rad) 
-> fwhm is 0.000000 arcmin
Sigma is 0.000000 arcmin (0.000000 rad) 
-> fwhm is 0.000000 arcmin
Sigma is 0.000000 arcmin (0.000000 rad) 
-> fwhm is 0.000000 arcmin
Sigma is 0.000000 arcmin (0.000000 rad) 
-> fwhm is 0.000000 arcmin
Sigma is 0.000000 arcmin (0.000000 rad) 
-> fwhm is 0.000000 arcmin
Sigma is 0.000000 arcmin (0.000000 rad) 
-> fwhm is 0.000000 arcmin
Sigma is 0.000000 arcmin (0.000000 rad) 
-> fwhm

/home/stagiaire/miniconda3/envs/py3/lib/python3.7/site-packages/healpy/sphtfunc.py:400: FutureChangeWarning: The order of the input cl's will change in a future release.
Use new=True keyword to start using the new order.
See documentation of healpy.synalm.
  category=FutureChangeWarning,


In [ ]:
moy_Maps = np.mean(Maps)
ecart_Maps = np.sqrt(np.var(Maps))
Max_Maps = np.abs(Maps).max()
Maps = (Maps-moy_Maps)/(np.abs(Maps).max())

In [ ]:
print(moy_Maps)
print(ecart_Maps)
print(np.abs(Maps).max())

## Préparation du ML

In [16]:
Ntrain = 10000-100
Ntest = 110
Max_l_p=np.abs(l_p).max()
NNl_p = (l_p-moy_l_p)/(np.abs(l_p).max())
X_train = Maps[:,0:(Ntrain)]
y_train = NNl_p[0:(Ntrain)]
X_test = Maps[:,(Ntrain):(Ntrain+Ntest)]
y_test = NNl_p[(Ntrain):(Ntrain+Ntest)]
print(y_train.shape, y_test.shape)
y_train = kr.utils.to_categorical(y_train)
y_test = kr.utils.to_categorical(y_test)
#y_test = y_test[:,:50]
#y_train = y_train.T
#y_test = y_test.T
print(y_train.shape, y_test.shape)
print(np.abs(NNl_p).max())

(90, 100, 2) (10, 100, 1)


In [17]:
seed = 7
np.random.seed(seed)
shape=(len(Maps[:,0]), 1)
num_classes = y_train.shape[1]
X_train = X_train.T
X_test = X_test.T
print(X_train.shape, shape, X_test.shape)
X_train = X_train.reshape(X_train.shape[0], len(X_train[0]), 1)
X_test = X_test.reshape(X_test.shape[0], len(X_test[0]), 1)
print(X_train.shape, shape, X_test.shape)

(90, 3072) (3072, 1) (10, 3072)
(90, 3072, 1) (3072, 1) (10, 3072, 1)


## NBB layers

In [17]:
inputs=kr.layers.Input(shape)
for i in range (int(math.log(nside,2))):
#   Recog of the neighbours & Convolution
    x = nnhealpix.layers.ConvNeighbours(int(nside/(2**(i))), filters=32, kernel_size=9)(inputs)
    x = kr.layers.Activation('relu')(x)
#   Degrade
    x = nnhealpix.layers.MaxPooling(int(nside/(2**(i))), int(nside/(2**(i+1))))(x)

x = kr.layers.Dropout(0.2)(x)
x = kr.layers.Flatten()(x)
x = kr.layers.Dense(128)(x)
x = kr.layers.Activation('relu')(x)
x = kr.layers.Dense(num_classes)(x)

out=kr.layers.Activation('softmax')(x)

IOError: [Errno 2] No such file or directory: '/home/stagiaire/.config/nnhealpix/ancillary_files/filter_nside16_order1.npz'

### Model

In [18]:
model = kr.models.Model(inputs=inputs, outputs=out)
model.compile(loss=kr.losses.mse, optimizer='adam', metrics=['accuracy'])
model.summary()

NameError: name 'out' is not defined

### Training the model

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=500, verbose=1)
scores = model.evaluate(X_test, y_test, verbose=0)
print("CNN Error: %.2f%%" % (100-scores[1]*100))

In [ ]:
prediction = model.predict(X_test)*ecart_l_p+moy_l_p